<a href="https://colab.research.google.com/github/virginiakm1988/Easy-Adapter/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parameter-efficient Fine-tuning in NLP

This code demonstrates how to fine-tune a BERT model based on the Hugging Face Transformers library with [Easy-Adapter](https://github.com/virginiakm1988/Easy-Adapter). Adapters are a parameter-efficient way to fine-tune a pre-trained language model for a specific NLP task.
This code demonstrates a practical example of using adapters in fine-tuning a BERT model. The code can be adapted to other pre-trained models and NLP tasks.

For any suggestions or questions, please contact Zih-Ching Chen (virginia.chen2007@gmail.com).

## Setup Instructions

Before running the code, please follow these setup instructions:

1. Install the necessary packages by running the following command: 

   ```
   ! pip install transformers datasets
   ! pip install loralib
   ```

2. Check that your system has a compatible GPU installed by running the following command in your terminal:

   ```
   nvidia-smi
   ```


Once you have completed these setup instructions, you are ready to run the code.

In [1]:
! pip install transformers datasets
! pip install loralib
! nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.1 MB/s eta 0:00:00
Looking in index

In [2]:
! git clone https://github.com/virginiakm1988/Easy-Adapter.git

Cloning into 'Easy-Adapter'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 115 (delta 55), reused 29 (delta 9), pack-reused 0
Receiving objects: 100% (115/115), 113.48 KiB | 3.24 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [3]:
%cd /content/Easy-Adapter/

/content/Easy-Adapter


## Define custom adapter modules
In [`adapters.py`](https://github.com/virginiakm1988/Easy-Adapter/blob/main/adapters.py), we implemented `Houlsby`, `ConvAdapters`, `AdapterBias`, and `LoRA`.
1. Houlsby Adapter ([Parameter-Efficient Transfer Learning for NLP](https://http://proceedings.mlr.press/v97/houlsby19a.html))
2. ConvAdapter ([CHAPTER: Exploiting Convolutional Neural Network Adapters for Self-supervised Speech Models](https://arxiv.org/abs/2212.01282))
3. AdapterBias ([AdapterBias: Parameter-efficient Token-dependent Representation Shift for Adapters in NLP Tasks](https://arxiv.org/abs/2205.00305))

4. LoRA ([LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685))

5. BitFit ([BitFit: Simple Parameter-efficient Fine-tuning for Transformer-based Masked Language-models](https://arxiv.org/abs/2106.10199)
BitFit can be implemented through the following settings:
```
mark_only_adapter_as_trainable(model_bert,bias="all")
```



## Adding Adapter to a Hugging Face Model


The code uses the `AutoModelForSequenceClassification` class to load a pre-trained BERT model (`bert-base-uncased` in this case). The code then sets the adapter type to "houlsby" and specifies the `lora_r` and `lora_alpha` parameters for the adapter.

The code then modifies each layer of the BERT encoder to include the adapter module. The output layer is modified using the `adapted_bert_output` function from the `adapter_bert` module, and the attention layer is modified using the `AdaptedBertSelfAttention` class from the same module.

Finally, the code freezes all parameters except for the adapter module by calling the `mark_only_adapter_as_trainable` function from the `utils` module.

In [22]:
from transformers import AutoModelForSequenceClassification
from adapter_bert import adapted_bert_output, AdaptedBertSelfAttention
from utils import mark_only_adapter_as_trainable
from torch import nn
import torch

BertLayerNorm = torch.nn.LayerNorm

model_bert = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
model_bert.config.adapter = "houlsby"
model_bert.config.lora_r = 8
model_bert.config.lora_alpha = 8
original_state_dict = model_bert.state_dict()
#add adapter module in a bert model
for idx, layer in enumerate(model_bert.bert.encoder.layer):
  #modify the output layer
  model_bert.bert.encoder.layer[idx].output = adapted_bert_output(model_bert.bert.encoder.layer[idx].output, model_bert.config)
  #modify the attention layer for adding lora
  model_bert.bert.encoder.layer[idx].attention.self = AdaptedBertSelfAttention(model_bert.bert.encoder.layer[idx].attention.self, model_bert.config)
model_bert.load_state_dict(original_state_dict,strict = False)
#freeze parameters
mark_only_adapter_as_trainable(model_bert)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Loading datasets
### Tokenizing an IMDb Dataset with BERT

This code demonstrates a practical example of loading and tokenizing a dataset using the Hugging Face Datasets and Transformers libraries. The resulting tokenized datasets can be used for fine-tuning a pre-trained language model on the IMDb sentiment classification task.

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("imdb")
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

## Define evaluation metric

In [ ]:
import numpy as np
def compute_metrics(p) :
    preds,labels=p
    preds = np.argmax(preds, axis=-1)
    #print('shape:', preds.shape, '\n')
    #precision, recall, f1, _ = precision_recall_fscore_support(lables.flatten(), preds.flatten(), average='weighted', zero_division=0)
    return {
        'accuracy': (preds == p.label_ids).mean(),

    }

## Training a BERT Model on IMDb Dataset

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir = "test-trainer",per_device_train_batch_size = 4)
trainer = Trainer(
    model=model_bert, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)
trainer.train() 

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.699000


TrainOutput(global_step=750, training_loss=0.6989628295898438, metrics={'train_runtime': 225.4509, 'train_samples_per_second': 13.307, 'train_steps_per_second': 3.327, 'total_flos': 813893437440000.0, 'train_loss': 0.6989628295898438, 'epoch': 3.0})

## Evaluate

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6928722262382507,
 'eval_runtime': 30.7938,
 'eval_samples_per_second': 32.474,
 'eval_steps_per_second': 4.059,
 'epoch': 3.0}

## Saving Adapter State Dictionary in a BERT Model


In [ ]:
from utils import adapter_state_dict
checkpoint_path = "./result"
torch.save(adapter_state_dict(model_bert), checkpoint_path)